In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.sonic.net/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xzf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

In [ ]:
df = spark.read.parquet("drive/MyDrive/Colab Notebooks/ejemplo.parquet")

In [ ]:
df.describe().toPandas()

,summary,timeID,deviceID,ClientID,date,sensors,macaddress,timestamp,lostPackages,transferid,temperature,humidity,pressure,rssi,battery,battid,battime
0,count,12960,12960,12960,12960,12960,12960,12960,12960,12960,12960,12960,12960,12960,12960,12960,12960
1,mean,None,None,None,None,92.0,None,1.6015538709233737E9,0.0,125.54969135802469,2173.533024691358,44.47091049382716,9509425.711111112,-54.885802469135804,2905.1457561728394,110.22445987654321,1.601481914506713E9
2,stddev,None,None,None,None,0.0,None,24941.957520820277,0.0,78.42305843142776,335.75949012408205,6.586593288428373,21137.794483612848,6.8812364478812365,88.72955972292038,3.1467937402000943,271386.6559376322
3,min,00:00,FB62FD5FCF937CE4,adamnavalcarnero,01/10/2020,92,c0636bd36671,1.60151097080094E9,0,0,1446,22,9442797,-76,2283,81,1598953146
4,max,23:50,FB62FD5FCF937CE4,adamnavalcarnero,02/10/2020,92,fee19f125bd3,1.60159677103228E9,0,255,3534,69,9557038,-34,2984,111,1601549966


In [ ]:
df.printSchema()

root
 |-- timeID: string (nullable = true)
 |-- deviceID: string (nullable = true)
 |-- ClientID: string (nullable = true)
 |-- date: string (nullable = true)
 |-- sensors: long (nullable = true)
 |-- macaddress: string (nullable = true)
 |-- timestamp: double (nullable = true)
 |-- lostPackages: long (nullable = true)
 |-- transferid: long (nullable = true)
 |-- temperature: long (nullable = true)
 |-- humidity: long (nullable = true)
 |-- pressure: long (nullable = true)
 |-- rssi: long (nullable = true)
 |-- battery: long (nullable = true)
 |-- battid: long (nullable = true)
 |-- battime: long (nullable = true)



In [ ]:
df.groupBy("macaddress").agg({"temperature":"min", "humidity": "mean"}).show()

+------------+----------------+-----------------+
|  macaddress|min(temperature)|    avg(humidity)|
+------------+----------------+-----------------+
|ce8fbc221d4f|            1683|49.05555555555556|
|da0e4a7275a5|            2350|36.69444444444444|
|ce57c6f3536a|            1821|46.84027777777778|
|d610970128dd|            2254|38.72222222222222|
|ce9ce9665b9b|            2116|          40.9375|
|cd4a1eb4b77b|            1446|54.27777777777778|
|fc45bd10fea1|            2207|39.80555555555556|
|c60bcd36bdde|            1700|          48.3125|
|d8a4615b5ac3|            2272|37.96527777777778|
|f578e67b1bed|            1838|          48.1875|
|d5a8c0f09911|            2332|36.27777777777778|
|db66e53d550c|            2158|41.88194444444444|
|f12ecbc4bb38|            2416|           36.875|
|e7186ed3bd76|            2172|40.99305555555556|
|c0c5b4353a4a|            1685|             51.0|
|c69105d8406c|            2218|41.40972222222222|
|ce084d0de0b2|            1877|46.56944444444444|


In [ ]:
type (df.select(["macaddress", "temperature"]))

pyspark.sql.dataframe.DataFrame

In [ ]:
type (df["macaddress"])

pyspark.sql.column.Column

In [ ]:
df.head(2)

[Row(timeID='02:00', deviceID='FB62FD5FCF937CE4', ClientID='adamnavalcarnero', date='01/10/2020', sensors=92, macaddress='c0c5b4353a4a', timestamp=1601510970.80094, lostPackages=0, transferid=9, temperature=1685, humidity=51, pressure=9535634, rssi=-69, battery=2283, battid=81, battime=1598953146),
 Row(timeID='02:00', deviceID='FB62FD5FCF937CE4', ClientID='adamnavalcarnero', date='01/10/2020', sensors=92, macaddress='c57a8f9365b6', timestamp=1601510970.80094, lostPackages=0, transferid=113, temperature=1811, humidity=51, pressure=9530229, rssi=-66, battery=2913, battid=110, battime=1601456004)]

In [ ]:
df2 = df.withColumn("realT", df["temperature"] / 100)

In [ ]:
df.groupBy("macaddress").agg({"temperature":"min", "humidity": "mean"}).withColumnRenamed("min(temperature)", "temp").show()

+------------+----+-----------------+
|  macaddress|temp|    avg(humidity)|
+------------+----+-----------------+
|ce8fbc221d4f|1683|49.05555555555556|
|da0e4a7275a5|2350|36.69444444444444|
|ce57c6f3536a|1821|46.84027777777778|
|d610970128dd|2254|38.72222222222222|
|ce9ce9665b9b|2116|          40.9375|
|cd4a1eb4b77b|1446|54.27777777777778|
|fc45bd10fea1|2207|39.80555555555556|
|c60bcd36bdde|1700|          48.3125|
|d8a4615b5ac3|2272|37.96527777777778|
|f578e67b1bed|1838|          48.1875|
|d5a8c0f09911|2332|36.27777777777778|
|db66e53d550c|2158|41.88194444444444|
|f12ecbc4bb38|2416|           36.875|
|e7186ed3bd76|2172|40.99305555555556|
|c0c5b4353a4a|1685|             51.0|
|c69105d8406c|2218|41.40972222222222|
|ce084d0de0b2|1877|46.56944444444444|
|c71fc208d39c|2045|          43.5625|
|c8fbf22cc044|1823|45.94444444444444|
|e17e519790db|1876|47.63194444444444|
+------------+----+-----------------+
only showing top 20 rows



In [ ]:
df.registerTempTable("datos")

In [ ]:
spark.sql("select temperature from datos where macaddress = 'ce8fbc221d4f'").show()

+-----------+
|temperature|
+-----------+
|       1918|
|       1903|
|       1890|
|       1881|
|       1881|
|       1876|
|       1876|
|       1876|
|       1876|
|       1843|
|       1823|
|       1823|
|       1829|
|       1829|
|       1829|
|       1829|
|       1840|
|       1840|
|       1825|
|       1820|
+-----------+
only showing top 20 rows



In [ ]:
df.filter(df["temperature"] > 3000).show()

+------+----------------+----------------+----------+-------+------------+-------------------+------------+----------+-----------+--------+--------+----+-------+------+----------+
|timeID|        deviceID|        ClientID|      date|sensors|  macaddress|          timestamp|lostPackages|transferid|temperature|humidity|pressure|rssi|battery|battid|   battime|
+------+----------------+----------------+----------+-------+------------+-------------------+------------+----------+-----------+--------+--------+----+-------+------+----------+
| 02:00|FB62FD5FCF937CE4|adamnavalcarnero|01/10/2020|     92|dbf2efd3de50| 1.60151097080094E9|           0|        38|       3354|      24| 9545278| -45|   2950|   110|1601456385|
| 02:00|FB62FD5FCF937CE4|adamnavalcarnero|01/10/2020|     92|ddac67f1b37f| 1.60151097080094E9|           0|       103|       3415|      22| 9536742| -66|   2984|   110|1601456471|
| 02:10|FB62FD5FCF937CE4|adamnavalcarnero|01/10/2020|     92|dbf2efd3de50|1.601511570801781E9|      

In [ ]:
spark.sql("select * from datos where temperature > 3000").show()

+------+----------------+----------------+----------+-------+------------+-------------------+------------+----------+-----------+--------+--------+----+-------+------+----------+
|timeID|        deviceID|        ClientID|      date|sensors|  macaddress|          timestamp|lostPackages|transferid|temperature|humidity|pressure|rssi|battery|battid|   battime|
+------+----------------+----------------+----------+-------+------------+-------------------+------------+----------+-----------+--------+--------+----+-------+------+----------+
| 02:00|FB62FD5FCF937CE4|adamnavalcarnero|01/10/2020|     92|dbf2efd3de50| 1.60151097080094E9|           0|        38|       3354|      24| 9545278| -45|   2950|   110|1601456385|
| 02:00|FB62FD5FCF937CE4|adamnavalcarnero|01/10/2020|     92|ddac67f1b37f| 1.60151097080094E9|           0|       103|       3415|      22| 9536742| -66|   2984|   110|1601456471|
| 02:10|FB62FD5FCF937CE4|adamnavalcarnero|01/10/2020|     92|dbf2efd3de50|1.601511570801781E9|      

In [ ]:
data = spark.read.options(header=True, inferSchema=True).csv("drive/MyDrive/Colab Notebooks/walmart_stock.csv")

In [ ]:
data.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

